In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

# 1. Loading Data and Fetures

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')
sample_submission = pd.read_csv("../input/tabular-playground-series-may-2022/sample_submission.csv")

In [ ]:
# from refence 1 notebook 
for df in [train, test]:
    for i in range(10):
        df[f'ch{i}'] = df.f_27.str.get(i).apply(ord) - ord('A')
    df["unique_characters"] = df.f_27.apply(lambda s: len(set(s)))
features = [f for f in test.columns if f != 'id' and f != 'f_27']
test[features].head(2)

In [ ]:
train.f_00[0:100].plot();
train.f_00[0:100].ewm(com=0.8,min_periods=1).mean().fillna(0).plot()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

# 2. Statistical Features
* mean : --,average at axis =1
* std : --,standart deviation at axis =1
* sum : --,summation at axis =1
* var : --,variance at axis =1
* euc : reduce_euclidean_norm,`Computes the Euclidean norm of elements across dimensions of a tensor.`[*](http://www.tensorflow.org/api_docs/python/tf/math/reduce_euclidean_norm)
* ptp : peak to peak,`Range of values (maximum - minimum) along an axis.`[**](https://numpy.org/doc/stable/reference/generated/numpy.ptp.html)
* logsumexp : log(sum(exp(value)))`Computes log(sum(exp(elements across dimensions of a tensor))).`[***](https://www.tensorflow.org/api_docs/python/tf/math/reduce_logsumexp)




In [ ]:
stat = ["mean","std","sum","var","euc","ptp","logsumexp"]

In [ ]:
cols = train.dtypes[train.dtypes==float].index.to_list()
def new_feats(df):
    
    df["mean"] = df[cols].mean(axis=1).values
    df["std"] = df[cols].std(axis=1).values
    df["sum"] = df[cols].sum(axis=1).values
    df["var"] = df[cols].var(axis=1).values
    df["euc"] =tf.math.reduce_euclidean_norm(df[cols],axis=1).numpy()
    df["ptp"] = df[cols].values.ptp(axis=1)
    df["logsumexp"] = tf.math.reduce_logsumexp(df[cols],axis=1).numpy()
    for col in cols:
        df[f"roll_{col}"] = df[col].ewm(com=0.8,min_periods=1).mean().fillna(0)
    
    
new_feats(train)
new_feats(test)

In [ ]:
rolls = [f for f in test.columns if f[0:4] == 'roll']

In [ ]:
scale_cols = [f for f in test.columns if train[f].dtype == float]

In [ ]:
## Standart scaling, standart scale only applied for float values, integers not scaled
sc=StandardScaler()
sc.fit(train[scale_cols].values)
scaled_st_train = sc.transform(train[scale_cols].values)
scaled_st_test = sc.transform(test[scale_cols].values)
train[scale_cols] = scaled_st_train
test[scale_cols] = scaled_st_test

In [ ]:
train[features+stat+rolls].values.reshape(8,8,train.shape[0],1).shape

In [ ]:
# reshape np.array for convolution
traincon = train[features+stat+rolls].values.reshape(train.shape[0],8,8)
testcon = test[features+stat+rolls].values.reshape(test.shape[0],8,8)

In [ ]:
plt.imshow(traincon[0])

In [ ]:
labels=train.target

In [ ]:
import random

## Plotting samples

In [ ]:
total_sample = 20
randInt = np.array([random.choice(labels[labels==1].index) for x in range(int(total_sample/2))]+
                        [random.choice(labels[labels==0].index) for x in range(int(total_sample/2)+1)])
fig,axs=plt.subplots(nrows=5,ncols=4,figsize=(20,20))
plt.subplots_adjust(wspace=-0.2, hspace=0.6)
for i, ax in enumerate(axs.flat):
    pcm = ax.imshow(traincon[randInt[i]].squeeze() ,cmap=plt.cm.magma_r)
    if labels[i]==0:
        ax.set_title(f"loc:{randInt[i]} state :{labels[i]}",fontdict={"color":"blue"})
    else :
        ax.set_title(f"loc:{randInt[i]} state :{labels[i]}",fontdict={"color":"green"})
    ax.set_xlabel("features",fontdict={"color":"green"})
    ax.set_ylabel("features",fontdict={"color":"green"})
    fig.colorbar(pcm, ax=ax)
    ax.set_xticks([])
    ax.set_yticks([])
plt.show()

In [ ]:
train_cols = test.columns.to_list()
train_cols.remove("id")

In [ ]:
len(features+stat+rolls)

In [ ]:
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.metrics import roc_curve
def plot_loss_auc(history,y_true,prediction):
    """
    history: history = model.fit() 
    y_true: true validation set or test set labels
    prediction: prediction on val set or test set
    """
    fp, tp, _ = roc_curve(y_true, prediction)
    _,ax = plt.subplots( ncols=4,nrows=1,figsize=(20,3))
    ax[1].set_xlabel("epochs")
    ax[1].set_ylabel("loss")
    ax[1].set_title("final val_loss %1.4f"%(history.history["val_loss"][-1:][0]))
    ax[2].set_xlabel("epochs")
    ax[2].set_ylabel("auc")
    ax[2].set_title("final val_auc %1.4f"%(history.history["val_auc"][-1:][0]))
    ax[0].set_xlabel("learning rate")
    ax[0].set_ylabel("loss")
    ax[0].set_title("semilogx lr vs loss")
    ax[3].plot(fp, tp,label="ROC", linewidth=2)
    ax[3].vlines(x=0,ymin=0.0,ymax=1.0,linewidth=0.5,color="r",linestyles="--")
    ax[3].hlines(y=1,xmin=0.0,xmax=1.0,linewidth=0.5,color="r",linestyles="--")
    ax[3].set_xlabel('False positives')
    ax[3].set_ylabel('True positives')
    ax[3].set_title("ROC")
    ax[0].semilogx(history.history["lr"], history.history["loss"])
    ax[0].set_ylim(ymax=0.11)
    pd.DataFrame([history.history["auc"],history.history["val_auc"]],index=["auc","val_auc"]).T.plot(ax=ax[2])
    pd.DataFrame([history.history["loss"],history.history["val_loss"]],index=["loss","val_loss"]).T.plot(ax=ax[1])
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
def plot_cm(y_true, prediction, p=0.5):
    """
    y_true: true validation set or test set labels
    prediction: prediction on val set or test set
    
    """
    cm = confusion_matrix(y_true, prediction > p)
    plt.figure(figsize=(3,3))
    sns.heatmap(cm, annot=True, fmt="d",cbar=False)
    plt.title('Confusion matrix @{:.2f}'.format(p))
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    
    print('\nState 0 Detected (True Negatives): ', cm[0][0])
    print('State 1 Incorrectly Detected (False Positives): ', cm[0][1])
    print('State 1 Missed (False Negatives): ', cm[1][0])
    print('State 1 Detected (True Positives): ', cm[1][1])
    print('Total States : ', np.sum(cm[1]))
    plt.show()

In [ ]:
#  this callback for verbose 0 console reporting, it doesn't dislay epoch steps.
class reps(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        
        print(f"epoch : {epoch}, loss : {logs.get('loss'):.4f}, val loss : {logs.get('val_loss'):.4f}, auc : {logs.get('auc'):.4f}, val auc : {logs.get('val_auc'):.4f}")
            

report_callback = reps()

## Ploting lrdecay 

In [ ]:
epochs = 50
exp=-0.05
lr=0.001 # lr at begin
start = 10 #epoch
mid = 17 #epoch
def lr_decay(epoch,lr):
            if epoch< start:
                return lr
            elif epoch<mid:
                return lr*(1 + np.cos(epoch / (epochs-1.5) * np.pi)) / 1.8 # from reference 1 notebook
            else:
                return lr * tf.math.exp(exp)


def plot_lr_decay(epochs,lr):
    x = np.arange(0,epochs)
    lrs = [] 
    lr2=lr
    for epoch in x:
        lr =  lr_decay(epoch,lr)
        lrs.append(lr)
    y = np.array(lrs)
    plt.figure(figsize=(8,4))
    plt.plot(x,y)
    plt.vlines(x=start-1,linestyles="--",colors="g",ymin=y[-1],ymax=lr2,linewidth=0.95)
    plt.vlines(x=mid-1,linestyles="--",colors="orange",ymin=y[-1],ymax=lr2,linewidth=0.95)
    plt.vlines(x=35,linestyles="--",colors="r",ymin=y[-1],ymax=lr2,linewidth=0.95)
    plt.hlines(y=0,linestyles="--",colors="r",xmin=start-1,xmax=50,linewidth=0.95)
    plt.xlabel("epochs")
    plt.ylabel("learning rate")
    plt.title("learning rate decay")
plot_lr_decay(epochs,lr)
lrDecay = tf.keras.callbacks.LearningRateScheduler(lr_decay) # lr decay callback
callbacks = [lrDecay,
            report_callback]

In [ ]:
from tensorflow.keras.layers import Conv1D,Conv2D,Flatten,Dropout,Conv1DTranspose,Conv2DTranspose,Dense,Reshape ,GlobalAveragePooling1D
from tensorflow.keras import Input
from tensorflow import keras


# Models

In [ ]:
# 2d + 1d convolutions this model used for training
lr = 0.001
input_shape = (8,8)
def cnn2():
    
    model = keras.Sequential(
    [
        Input(shape=input_shape),
        Reshape(target_shape=(8, 8,1)),
        Conv2D(
            filters=144, kernel_size=(7,7), padding="same", strides=2, activation="relu"
        ),
        Dropout(rate=0.1),
        Reshape(target_shape=(4*12, 4*12)),
        Conv1D(
            filters=64, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        Conv1DTranspose(
            filters=36, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
#         
        Conv1DTranspose(
            filters=64, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        Dropout(rate=0.1),
        Conv1DTranspose(
            filters=128, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        Flatten(),
        Dropout(rate=0.4),
        Dense(1,activation="sigmoid")
        ]
    )   
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer = optimizer , loss = "binary_crossentropy", metrics = ["AUC"])
    return model

In [ ]:
# 1d convolutions 
lr = 0.001
input_shape = (8,8)
def cnn():
    
    model = keras.Sequential(
    [
        Input(shape=input_shape),
#         Reshape(target_shape=(8, 8,1)),
        Conv1D(
            filters=128, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        Dropout(rate=0.2),
#         Reshape(target_shape=(4*12, 4*12)),
        Conv1D(
            filters=64, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        Conv1DTranspose(
            filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
#         
        Conv1DTranspose(
            filters=64, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        Dropout(rate=0.2),
        Conv1DTranspose(
            filters=128, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
#         Conv1DTranspose(filters=3, kernel_size=7, padding="same"),
        Flatten(),
#         Dropout(rate=0.4),
        Dense(1,activation="sigmoid")
        ]
    )   
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer = optimizer , loss = "binary_crossentropy", metrics = ["AUC"])
    return model

In [ ]:
model = cnn2()
model.summary()

In [ ]:
import scipy.stats
from sklearn.model_selection import KFold,train_test_split

# Training

In [ ]:
epochs = 45
batch_size = 32
pred_list = []
cv = 2 # min value 2 for fold split
verbose = 0 
kf = KFold(n_splits=cv)
fold_split = False # {True: KFold split | False: train_test_split} 
if fold_split:
    for fold, (split_train, split_test) in enumerate(kf.split(train)):
        print("\n","*="*10+"*",f"fold {fold+1}","*="*10+"*","\n")
        X_train = traincon[split_train]
        X_test = traincon[split_test]
        y_train = labels[split_train]
        y_test = labels[split_test]

        with strategy.scope():
            model = cnn2()
            history = model.fit(X_train, y_train,batch_size=batch_size,epochs=epochs,callbacks=callbacks,
                                validation_data=(X_test,y_test),verbose=verbose,shuffle=True,steps_per_epoch = X_train.shape[0]//batch_size)
        pred_list.append(scipy.stats.rankdata(model.predict(testcon)))
        pred_nn = ((model.predict(X_test).reshape(1,-1)[0])>0.5).astype(int)
        plot_loss_auc(history,y_test,pred_nn)
        plot_cm(y_test,pred_nn, p=0.5)

else: # train_test_split
    random_states=np.linspace(789,9876,cv).astype(int)
    print("random_states:",random_states)
    for fold,random_state in zip(range(cv),random_states):
        print("\n","*="*10+"*",f"fold {fold+1}","*="*10+"*","\n")
        X_train, X_test, y_train, y_test = train_test_split(traincon,labels,test_size=0.06,random_state=random_state)
        with strategy.scope():
            model = cnn2()
            history = model.fit(X_train, y_train,batch_size=batch_size,epochs=45,callbacks=callbacks,
                                validation_data=(X_test,y_test),verbose=verbose,shuffle=True,steps_per_epoch = X_train.shape[0]//batch_size)
        pred_list.append(scipy.stats.rankdata(model.predict(testcon)))
        pred_nn = ((model.predict(X_test).reshape(1,-1)[0])>0.5).astype(int)
        plot_loss_auc(history,y_test,pred_nn)
        plot_cm(y_test,pred_nn, p=0.5)


# Submission

In [ ]:

mean_cnn = np.array(pred_list).mean(axis=0).reshape(1,-1)[0]

In [ ]:
sample_submission['target'] = mean_cnn
sample_submission.to_csv('submission.csv', index=False)
sample_submission

# References
<p></p>

[ref 1. tpsmay22-keras-quickstart from AMBROSM](https://www.kaggle.com/code/ambrosm/tpsmay22-keras-quickstart)